![](python_snake.png)

### Python Time : the ```datetime``` module
[Link](https://docs.python.org/3/library/datetime.html)

- [UTC](https://en.wikipedia.org/wiki/Coordinated_Universal_Time) (Coordinated Universal Time) : the time at longitude 0 (No daylight savings time)

- [Unix time](https://en.wikipedia.org/wiki/Unix_time) - the number of seconds since Jan 1, 1970

- [ISO 8601 format](https://en.wikipedia.org/wiki/ISO_8601) : numbers run from most to least significant, left to right.


In [35]:
from datetime import datetime # standard library also

print(f'Unix time : {datetime.now().timestamp()}\n') # Unix time

print(f'datetime.now() : {datetime.now()}\n') # Year, month, day, hour, minute, second, nanoseconds

print(f'isoformat : {datetime.now().isoformat()}') # ISO 8601 format : most to least significant figures, LTR

Unix time : 1743086367.049132

datetime.now() : 2025-03-27 14:39:27.049244

isoformat : 2025-03-27T14:39:27.049309


#### Often, we want to convert a datetime object to a human readable string

For this we can use datetime.strftime()

In [36]:
now = datetime.now()
print(now)

datetime.strftime(now, '%a, %d %B %Y')

2025-03-27 14:39:27.055005


'Thu, 27 March 2025'

We can convert in the opposite direction using datetime.strptime(). Here's a [link](https://docs.python.org/3/library/datetime.html#format-codes) to the docs for the format codes

In [37]:
date = datetime.strptime('Mon, 4 April 2025', '%a, %d %B %Y')
date

datetime.datetime(2025, 4, 4, 0, 0)

datetime.strptime() is highly sensitive to the format - look what happens if we leave out the ','

In [38]:
date = datetime.strptime('Mon 4 April 2025',  '%a, %d %B %Y')
date

ValueError: time data 'Mon 4 April 2025' does not match format '%a, %d %B %Y'

There is a third party library, python-dateutil, that can help with parsing date strings.

If the import fails, you can install it with ```pip install python-dateutil && pip freeze --local > requirements.txt```

In [ ]:
from dateutil.parser import parse

print(parse('Mon 4 April 2025')) # our example from the previous code cell
print(parse('Monday, 21st June 25'))
print(parse("4th of April, '25"))

2025-04-04 00:00:00
2025-06-21 00:00:00
2025-04-04 00:00:00


#### Performing arithmetic with datetime objects

Once we have converted a string into a datetime object, we can then perform arithmetical operations

In [ ]:
start_date = datetime(year=2025, month=3, day=28, hour=15, minute=30)
end_date = datetime(year=2026, month=1, day=3)

print(f'difference : {end_date - start_date}')
print(type(end_date - start_date))

difference : {end_date - start_date}
<class 'datetime.timedelta'>


In [ ]:
from datetime import timedelta

now = datetime.now()
time_offset = timedelta(days=14)

in_a_fortnight = now + time_offset
datetime.strftime(in_a_fortnight, '%A, %d %B %Y')



'Thursday, 10 April 2025'

#### And we can use the comparison operators too

In [ ]:
start_date = datetime(year=2025, month=3, day=4)
end_date = datetime(year=2096, month=1, day=3)

now = datetime.now()

start_date <= now <= end_date

True

## Pandas time

![](panda_w_computer.png)

#### Pandas has an equivalent data type for datetime objects - [Timestamp](https://pandas.pydata.org/docs/reference/api/pandas.Timestamp.html)

It allows some of the same operations as python's datetime.

In [ ]:
import pandas as pd

date = pd.Timestamp(year=2025, month=4, day=23)

Timestamp('2025-04-23 00:00:00')

We can use ```strftime()```, but not ```strptime()```

In [ ]:
pd.Timestamp.strftime(date, '%A, %d %B %Y')

'Friday, 04 April 2025'

In [ ]:
parsed_date = pd.Timestamp.strptime('Monday, 3 June 2034', '%A, %d %B %Y')

NotImplementedError: Timestamp.strptime() is not implemented. Use to_datetime() to parse date strings.

If we want to parse strings into dates

And we can compare Timestamps

In [39]:
from_date = pd.Timestamp(year=2015, month=12, day=31) 
to_date = pd.Timestamp(year=2096, month=3, day=31) # pandas Timestamp

today = pd.Timestamp.now() # 

from_date < today < to_date

True

We can perform arithmetic that combines python datetimes and pandas Timestamps

In [ ]:
time_offset = timedelta(days=14) # python timedelta
today + time_offset

Timestamp('2025-04-10 14:15:25.318503')

And compare them

In [ ]:
today_python = datetime.now() # python now
from_date < today_python < to_date # comparing pandas from/to to python now

True

If we have a column in a csv file that represents a date, we can attempt to parse it into a Timestamp using ```pd.to_datetime``` .... here's the [documentation](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html#pandas.to_datetime)

In [40]:
df = pd.read_csv('customer_shopping_data.csv')
df.head()

,invoice_no,customer_id,gender,age,category,quantity,price,payment_method,invoice_date,shopping_mall
0,I138884,C241288,Female,28,Clothing,5,1500.40,Credit Card,5/8/22,Kanyon
1,I317333,C111565,Male,21,Shoes,3,1800.51,Debit Card,12/12/21,Forum Istanbul
2,I127801,C266599,Male,20,Clothing,1,300.08,Cash,9/11/21,Metrocity
3,I173702,C988172,Female,66,Shoes,5,3000.85,Credit Card,16/05/2021,Metropol AVM
4,I337046,C189076,Female,53,Books,4,60.60,Cash,24/10/2021,Kanyon


In [ ]:
df['datetime'] = pd.to_datetime(df['invoice_date'], errors='raise')
df['datetime'].head(3)

/tmp/ipykernel_1275/3323870671.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['datetime'] = pd.to_datetime(df['invoice_date'], errors='raise')


0   2022-05-08
1   2021-12-12
2   2021-09-11
Name: datetime, dtype: datetime64[ns]

#### We can try throwing a spanner in the works by re-writing the date format in one row

In [44]:
df.loc[3, 'invoice_date'] = 'Monday, 1st April 2025'
df.head()

,invoice_no,customer_id,gender,age,category,quantity,price,payment_method,invoice_date,shopping_mall,datetime
0,I138884,C241288,Female,28,Clothing,5,1500.40,Credit Card,5/8/22,Kanyon,2022-05-08
1,I317333,C111565,Male,21,Shoes,3,1800.51,Debit Card,12/12/21,Forum Istanbul,2021-12-12
2,I127801,C266599,Male,20,Clothing,1,300.08,Cash,9/11/21,Metrocity,2021-09-11
3,I173702,C988172,Female,66,Shoes,5,3000.85,Credit Card,"Monday, 1st April 2025",Metropol AVM,2025-04-01
4,I337046,C189076,Female,53,Books,4,60.60,Cash,24/10/2021,Kanyon,2021-10-24


As you can see from the UserWarning when we run the next code cell, the ```pd.to_datetime()``` function falls back to using the ```dateutil``` module mentioned above when it encounters our rogue format

In [45]:
df['datetime'] = pd.to_datetime(df['invoice_date'], errors='raise')
df['datetime'].head(10)

/tmp/ipykernel_1275/1525331766.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['datetime'] = pd.to_datetime(df['invoice_date'], errors='raise')


0   2022-05-08
1   2021-12-12
2   2021-09-11
3   2025-04-01
4   2021-10-24
5   2022-05-24
6   2022-03-13
7   2021-01-13
8   2021-04-11
9   2021-08-22
Name: datetime, dtype: datetime64[ns]

And parses the date correctly! We can specify this behaviour in advance by passing the format parameter as 'mixed'

In [46]:
df['datetime'] = pd.to_datetime(df['invoice_date'], errors='raise', format='mixed')
df['datetime'].head(10)

0   2022-05-08
1   2021-12-12
2   2021-09-11
3   2025-04-01
4   2021-10-24
5   2022-05-24
6   2022-03-13
7   2021-01-13
8   2021-04-11
9   2021-08-22
Name: datetime, dtype: datetime64[ns]